In [ ]:
# !pip install together

In [ ]:
from together import Together
import os

In [ ]:
roles = ['Data Scientist' , 'Data Analyst' , 'Data Engineer' , 'Software Engineer' , 'Machine Learning Engineer']
performance = ['select','reject']
rejection_reason = ['Insufficient technical knowledge for the required role',
                    'Lack of relevant industry experience',
                    'Poor communication or interpersonal skills',
                    'Inability to solve technical or problem-solving tasks during interviews',
                    'Misalignment with company culture or values',
                    'Inadequate understanding of job responsibilities or domain expertise',
                    'Overqualification or salary expectations beyond the budget',
                    'Failure to provide strong references or credible past work evidence',
                    'Lack of enthusiasm or motivation for the role',
                    'Limited adaptability to work in a team or dynamic environments']
selection_reasons = [
    'Demonstrated strong technical knowledge and expertise in the required domain',
    'Relevant and extensive industry experience aligned with the role',
    'Excellent communication and interpersonal skills',
    'Exceptional problem-solving abilities during technical assessments',
    'Alignment with company culture and values',
    'Clear understanding of job responsibilities and domain expertise',
    'Flexibility and adaptability to dynamic work environments',
    'Proven track record and strong references from past roles',
    'Enthusiasm and motivation to contribute to the company’s goals',
    'Effective team collaboration and leadership potential'
]

In [ ]:
os.environ['TOGETHER_API_KEY'] = 'cd9e34c00f76d13a8b55b3d34e48f782ac6ac0529cf14bf819a6b88d38d11cd0'
client = Together()

In [ ]:
job_descriptions = {role:[] for role in roles}
for job in roles:
  for i in range(5):
      response = client.chat.completions.create(
                          model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
                          messages=[{"role": "user", "content": f"Can u generate Job description for {job} role by assuming some required details and which is not generated previously, dont generate any text other than the job description like here is the job description ?"}],)
      job_descriptions[job].append(response.choices[0].message.content)
  job_descriptions[job] = list(set(job_descriptions[job]))

In [ ]:
print(job_descriptions['Software Engineer'][1])

Job Title: Software Engineer - Cloud Infrastructure
Reports to: Technical Lead, Cloud Engineering
Location: New York, USA
Job Type: Full-time

Job Summary:
We are seeking an experienced Software Engineer to join our Cloud Engineering team, responsible for designing, developing, and maintaining our cloud-based infrastructure. The ideal candidate will have a strong background in software development, cloud computing, and a passion for innovation.

Key Responsibilities:
- Design, develop, and deploy scalable, secure, and efficient cloud-based systems using AWS and Azure
- Collaborate with cross-functional teams to identify and prioritize project requirements
- Develop and maintain software applications, tools, and scripts to support cloud infrastructure
- Troubleshoot and resolve complex technical issues, providing root cause analysis and recommendations for improvement
- Participate in code reviews, ensuring high-quality, readable, and well-documented code
- Stay up-to-date with industry

In [ ]:
# data = {'Role' : [] , 'Transcript' : [] ,'Resume' : [] , 'Performance(select/reject)':[] ,'Reason for decision':[],'Job Description':[]}
# for job in roles:
#   for perf in performance:
#     if perf == 'reject':
#       for reason in rejection_reason:
#         for desc in job_descriptions[job]:
#           prompt = f'Can u generate the transcript between the interviewer and interviewee for the role of {job} such that the candidate got {perf} and the reason for rejection should be {reason} and the description of the job looks like {desc}'
#           response = client.chat.completions.create(
#                           model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
#                           messages=[{"role": "user", "content": prompt}],)
#           data[prompt] = response.choices[0].message.content
#     else :
#       for reason in selection_reasons:
#         for desc in job_descriptions[job]:
#           pass

In [ ]:
import time
import pandas as pd

In [ ]:
df = pd.DataFrame(columns=['Role', 'Transcript', 'Resume', 'Performance(select/reject)', 'Reason for decision', 'Job Description'])

In [ ]:
df.head()

,Role,Transcript,Resume,Performance(select/reject),Reason for decision,Job Description


In [ ]:
# Synthetic Data Generation

for job in roles:
    for perf in performance:
        if perf == 'reject':
            for reason in rejection_reason:
                for desc in job_descriptions[job]:
                    # Prompt for generating transcript
                    transcript_prompt = (
                        f"Can you generate the transcript between the interviewer and interviewee for the role of {job} "
                        f"such that the candidate got {perf} and the reason for rejection should be {reason} "
                        f"and the description of the job looks like {desc}"
                        "Don't generate any text apart from the required thing"
                    )
                    response = client.chat.completions.create(
                        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
                        messages=[{"role": "user", "content": transcript_prompt}],
                    )
                    transcript = response.choices[0].message.content

                    # Prompt for generating resume
                    resume_prompt = (
                        f"Can you generate a resume for a candidate applying for the role of {job} who was rejected? "
                        f"The rejection reason is {reason}, and the job description looks like: {desc}. "
                        f"The resume should reflect a typical candidate profile for this job and the rejection reason."
                        "Don't generate any text apart from the required thing"
                    )
                    resume_response = client.chat.completions.create(
                        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
                        messages=[{"role": "user", "content": resume_prompt}],
                    )
                    resume = resume_response.choices[0].message.content


                    row = {
                              'Role': job,
                              'Transcript': transcript,
                              'Resume': resume,
                              'Performance(select/reject)': perf,
                              'Reason for decision': reason,
                              'Job Description': desc
                            }

                    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

                    # Print and wait to prevent exceeding per-second requests
                    print(f"Processed rejection for {job} with reason: {reason}")
                    time.sleep(1)  # Sleep for 1 second to limit request rate

        else:  # For "select" performance
            for reason in selection_reasons:
                for desc in job_descriptions[job]:
                    # Prompt for generating transcript
                    transcript_prompt = (
                        f"Can you generate the transcript between the interviewer and interviewee for the role of {job} "
                        f"such that the candidate got {perf} and the reason for selection should be {reason} "
                        f"and the description of the job looks like {desc}"
                        "Don't generate any text apart from the required thing"
                    )
                    response = client.chat.completions.create(
                        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
                        messages=[{"role": "user", "content": transcript_prompt}],
                    )
                    transcript = response.choices[0].message.content

                    # Prompt for generating resume
                    resume_prompt = (
                        f"Can you generate a resume for a candidate applying for the role of {job} who was selected? "
                        f"The selection reason is {reason}, and the job description looks like: {desc}. "
                        f"The resume should reflect a typical candidate profile for this job and the selection reason."
                        "Don't generate any text apart from the required thing"
                    )
                    resume_response = client.chat.completions.create(
                        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
                        messages=[{"role": "user", "content": resume_prompt}],
                    )
                    resume = resume_response.choices[0].message.content

                    row = {'Role': job,'Transcript': transcript,'Resume': resume,'Performance(select/reject)': perf,'Reason for decision': reason,'Job Description': desc}
                    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

                    # Print and wait to prevent exceeding per-second requests
                    print(f"Processed selection for {job} with reason: {reason}")
                    time.sleep(1)  # Sleep for 1 second to limit request rate


Processed selection for Data Scientist with reason: Demonstrated strong technical knowledge and expertise in the required domain
Processed selection for Data Scientist with reason: Demonstrated strong technical knowledge and expertise in the required domain
Processed selection for Data Scientist with reason: Demonstrated strong technical knowledge and expertise in the required domain
Processed selection for Data Scientist with reason: Demonstrated strong technical knowledge and expertise in the required domain
Processed selection for Data Scientist with reason: Demonstrated strong technical knowledge and expertise in the required domain
Processed selection for Data Scientist with reason: Relevant and extensive industry experience aligned with the role
Processed selection for Data Scientist with reason: Relevant and extensive industry experience aligned with the role
Processed selection for Data Scientist with reason: Relevant and extensive industry experience aligned with the role
Proc

In [ ]:
df.to_csv("KasaPavan_SyntheticData.csv")

In [ ]:
print(df['Transcript'][1])

Interviewer: Good morning, welcome to the interview for the Data Scientist position. Can you start by telling me a little about yourself and why you're interested in this role?

Interviewee: Good morning, thank you for having me. With a Master's degree in Computer Science and over 4 years of experience in data science, I'm confident in my ability to develop and implement advanced analytics and machine learning models. I'm excited about this role because it offers the opportunity to work on complex business problems and drive growth through data-driven decision-making.

Interviewer: That's great. Can you walk me through your experience with machine learning frameworks and libraries? How have you applied them in previous roles?

Interviewee: I have experience with scikit-learn, TensorFlow, and PyTorch. In my previous role, I developed a predictive model using scikit-learn that increased sales forecast accuracy by 25%. I've also worked with TensorFlow to develop a recommender system that 